In [2]:
from dotenv import load_dotenv

import mlflow
from mlflow.models import infer_signature
from mlflow.data.pandas_dataset import PandasDataset

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

load_dotenv("../.env")

True

In [3]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True, as_frame=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
X_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4
65,6.7,3.1,4.4,1.4
11,4.8,3.4,1.6,0.2
42,4.4,3.2,1.3,0.2
...,...,...,...,...
71,6.1,2.8,4.0,1.3
106,4.9,2.5,4.5,1.7
14,5.8,4.0,1.2,0.2
92,5.8,2.6,4.0,1.2


In [5]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [13]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 10,
    "multi_class": "multinomial",
    "random_state": 2,
}

In [14]:
# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

/home/ono/otus/ml-advanced/otus-mlflow-local/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/ono/otus/ml-advanced/otus-mlflow-local/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10, multi_class='multinomial', random_state=2)

In [15]:
# Predict on the test set
y_pred_lr = lr.predict(X_test)

In [16]:
# Calculate metrics
accuracy_lr = accuracy_score(y_test, y_pred_lr)
accuracy_lr

1.0

In [17]:
import os

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")

In [ ]:
# os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI
# os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9090"

In [18]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)

# Create a new MLflow Experiment
mlflow.set_experiment("Quickstart Notebook Experiment")

<Experiment: artifact_location='s3://mlflow/1', creation_time=1762540091254, experiment_id='1', last_update_time=1762540091254, lifecycle_stage='active', name='Quickstart Notebook Experiment', tags={}>

In [ ]:
# Start an MLflow run
with mlflow.start_run(run_name='First run'):
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("LogisticRegression accuracy", accuracy_lr)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="notebook-tracking-quickstart",
    )

    dataset: PandasDataset = mlflow.data.from_pandas(X_train)
    mlflow.log_input(dataset, context="training")

2025/11/07 21:34:34 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'notebook-tracking-quickstart' already exists. Creating a new version of this model...
2025/11/07 21:34:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: notebook-tracking-quickstart, version 2


🏃 View run First run at: http://localhost:5000/#/experiments/1/runs/cda43ef913e44445bf70f158a8a69180
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '2' of model 'notebook-tracking-quickstart'.
